# 실습예제 : 미국 농무부 음식 데이터베이스 

## step 1. 데이터 가져오기

In [ ]:
import json
import urllib
from pandas import DataFrame, Series
import pandas as pd

url = 'https://github.com/pydata/pydata-book/blob/master/ch07/foods-2011-10-03.json?raw=true'
r = urllib.urlopen(url)
db = json.load(r)

len(db)

In [ ]:
db[0].keys()

In [ ]:
db[0]['nutrients'][0]

In [ ]:
nutrients = DataFrame(db[0]['nutrients'])

In [ ]:
nutrients[:7]

## step 2. 음식의 이름과 그룹, id 그리고 제조사 추출

In [ ]:
info_keys = ['description', 'group', 'id', 'manufacturer']
info = DataFrame(db, columns=info_keys)

In [ ]:
info[:5]

In [ ]:
info

## step 3. 음식 그룹의 분포 확인

In [ ]:
pd.value_counts(info.group)[:10]

## step 4. 영양소 정보 분석

In [ ]:
nutrients = []

for rec in db : 
    fnuts = DataFrame(rec['nutrients'])
    fnuts['id'] = rec['id']
    nutrients.append(fnuts)   


In [ ]:
nutrients[:2]

In [ ]:
nutrients = pd.concat(nutrients, ignore_index=True)

In [ ]:
nutrients

In [ ]:
nutrients.duplicated().sum()

In [ ]:
nutrients = nutrients.drop_duplicates()

In [ ]:
col_mapping = {'description' : 'food', 
                       'group' : 'fgroup'}

info = info.rename(columns = col_mapping, copy=False)
info

In [ ]:
col_mapping = {'description' : 'nutrient',
                       'group' : 'nutgroup'}
nutrients = nutrients.rename(columns=col_mapping, copy=False)

nutrients

In [ ]:
ndata = pd.merge(nutrients, info, on='id', how='outer')

In [ ]:
ndata

In [ ]:
ndata.ix[30000]

## step 5. 시각화    

In [ ]:
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)

In [ ]:
%pylab inline

In [ ]:
result['Zinc, Zn'].order().plot(kind='barh')

In [ ]:
by_nutrient = ndata.groupby(['nutgroup', 'nutrient'])

get_maximum = lambda x: x.xs(x.value.idxmax())
get_minimum = lambda x: x.xs(x.value.idxmin())

max_foods = by_nutrient.apply(get_maximum)[['value', 'food']]

max_foods.food = max_foods.food.str[:50]

In [ ]:
max_foods.ix['Amino Acids']['food']